In [1]:
import tensorflow as tf

import keras

import numpy as np

/home/anton/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [8]:
from keras.layers import Input, LSTM, RepeatVector, Dense, Embedding, Activation, Masking, SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.models import Model, Sequential
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [3]:
from functools import reduce


## Data

In [4]:
X = np.load("./spacers_as_array.npy")
Y = np.load("./spacers_as_OH_array.npy")
print(X.shape, Y.shape)

(1752, 51) (1752, 51, 7)


In [41]:
VOC_SIZE = 6
mask_val = 0
out_dim = 20
LSTM_INTERNAL_DIM = 200

## NET

In [34]:
#ENCODER 
encoder_input = Input(shape=(None,))

encoder_embedding_layer = Embedding(VOC_SIZE + 1, out_dim, mask_zero=True)
encoder_embedding = encoder_embedding_layer(encoder_input)

encoder_recurent_layer_1 = LSTM(LSTM_INTERNAL_DIM, return_state=True, return_sequences=True )
encoder_recurent_1, encoder_h, encoder_c = encoder_recurent_layer_1(encoder_embedding)

encoder_out_layer = TimeDistributed(Dense(VOC_SIZE, activation='softmax'))
encoder_out = encoder_out_layer(encoder_recurent_1)


#DECODER 
decoder_input = Input(shape=(None,))

decoder_embedding = encoder_embedding_layer(decoder_input)

decoder_recurent_layer_1 = LSTM(LSTM_INTERNAL_DIM, return_sequences=True, return_state=True)
decoder_recurent_1, _, _  = decoder_recurent_layer_1(decoder_embedding, initial_state = [encoder_h, encoder_c])

decoder_out_layer = TimeDistributed(Dense(VOC_SIZE, activation='softmax'))
decoder_out = decoder_out_layer(decoder_recurent_1)

In [35]:
encoder = Model(encoder_input, [encoder_h, encoder_c])

total_model = Model([encoder_input, decoder_input], decoder_out)

In [36]:
filepath_weights="weights-improvement.hdf5"
filepath_loss="loss-all-{epoch:02d}.hdf5"

In [37]:
weights_monitor = ModelCheckpoint(filepath_weights, monitor = 'loss', save_best_only=True, save_weights_only=True)
#loss_monitor = ModelCheckpoint(filepath_loss, monitor = 'loss', save_best_only=True, save_weights_only=True)

In [38]:
callbacks_list = [weights_monitor, loss_monitor]

In [39]:
total_model.compile(optimizer='adam', loss='categorical_crossentropy')

In [40]:
learning_progress = total_model.fit([X[:,:], X[:,:-1]], Y[:,1:,1:],
                                    batch_size=20,
                                    epochs=100,
                                    validation_split=0,callbacks = callbacks_list)

Epoch 1/100
1752/1752 [==============================] - 10s 6ms/step - loss: 1.4428
Epoch 2/100
1752/1752 [==============================] - 8s 5ms/step - loss: 1.3295
Epoch 3/100
1752/1752 [==============================] - 8s 4ms/step - loss: 1.3185
Epoch 4/100
1700/1752 [============================>.] - ETA: 0s - loss: 1.3132

KeyboardInterrupt: 